In [227]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns


import requests
import csv
import json

import warnings

In [228]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")
NASDAQ_DATA_LINK_API_KEY = os.getenv("NASDAQ_DATA_LINK_API_KEY")

In [229]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [254]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# Define the ticker symbols as a list; eg. TSM,MSFT,AMZN
# ticker_symbols = input("Enter stock tickers separated by commas:") 
# ticker_symbols = ticker_symbols.split(',')
ticker_symbols = [

    # energy
    'SLB'
    # ,'HAL'
    # ,'OKE'

    # consumer staples
    # ,'TGT'
    # ,'STZ'
    # ,'DLTR'
    # ,'DG'
    # ,'TSN'
    # ,'MNST'
    # ,'PG'
    # ,'PM'
    # ,'MO'


    # finanace
    # ,'TRV'
    # ,'CB'
    # ,'BAC'

    # technology
    # ,'NVDA'
    ,'MSFT'

    # INDEX
    # ,'SPY'
    # ,'QQQ'

    
    # holding stock
    ,'TSM'
    ,'ACN'
    ,'DHI'
    # ,'STZ'
    ,'GOOG'
    # ,'TGT'
]
# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 6 # this will return x-1 yr PE range

ticker_dict_json = {}
ticker_dict_pd = {}
ticker_screen_df = pd.DataFrame(
    columns=[x for x in range(0,26)] # 25 elements to be renamed
) 

# PE TTM Valuation

In [238]:
# Daily quote section
for i, symbol in enumerate(ticker_symbols):


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': f'{symbol}'
                    }
                ,inplace=True
                )
            
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].round(2)


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': f'{symbol}'
            }
        ,inplace=True
        )

    Monthly_stock_df[f'{symbol}'] = Monthly_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
    Monthly_stock_df[f'{symbol}'] = Monthly_stock_df[f'{symbol}'].round(2)



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - 6) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue

            annualEPS_df[f'{symbol}_PE'] = Monthly_stock_df[f'{symbol}'].values / annualEPS_df['reportedEPS'].values
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"{symbol}_PE"].mean().round(2)
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df[f"{symbol}_PE"]).round(2)
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[f'{col}'] = qtrEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from yf API
    yf_data = yf.Ticker(symbol).info

    forecast_earnings_keys = [
        'forwardEps'
        ,'forwardPE'
        ,'pegRatio'
    ]


    if not all(key in yf_data.keys() for key in forecast_earnings_keys):
        # Handle the case where one or more keys are missing
        EPS_12month_projected = 0
        PE_12month_projected = 0
        PEG_12month_projected = 0
    else:
        EPS_12month_projected = yf_data['forwardEps'] # 代表了截止下一个日历年结束的EPS, next year forecasted EPS
        PE_12month_projected = yf_data['forwardPE'] # 代表了截止下一个日历年结束的PE, next year forecasted PE
        PEG_12month_projected = yf_data['pegRatio'] # 代表了截止下一个日历年结束的PEG 


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]




    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate']<= i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, f"{symbol}_EPS_TTM"] = EPS_TTM

        if i == max(stock_consolidate_df.index):
            EPS_latest_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df[f"{symbol}_EPS_latest_projected"] = EPS_latest_projected
    

    # stock's stats
    stock_consolidate_df[f"{symbol}_PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df[f"{symbol}_PE_TTM"] = (stock_consolidate_df[symbol] / stock_consolidate_df[f"{symbol}_EPS_TTM"]).round(2)
    stock_consolidate_df[f"{symbol}_PE_TTM_avg"] = stock_consolidate_df[f"{symbol}_PE_TTM"].mean().round(2)
    stock_consolidate_df[f"{symbol}_PE_TTM_std"] = np.std(stock_consolidate_df[f"{symbol}_PE_TTM"]).round(2)
    stock_consolidate_df[f"{symbol}_PE_TTM_volatility_+"] = (stock_consolidate_df[f"{symbol}_PE_TTM_avg"] + stock_consolidate_df[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df[f"{symbol}_PE_TTM_volatility_-"] = (stock_consolidate_df[f"{symbol}_PE_TTM_avg"] - stock_consolidate_df[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df[f"{symbol}_relative_valuation_TTM_+"] = (stock_consolidate_df[f"{symbol}_PE_TTM_volatility_+"] * stock_consolidate_df[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df[f"{symbol}_relative_valuation_TTM_-"] = (stock_consolidate_df[f"{symbol}_PE_TTM_volatility_-"] * stock_consolidate_df[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df[f"{symbol}_relative_valuation_TTM_median"] = (np.median([stock_consolidate_df[f"{symbol}_relative_valuation_TTM_+"][0], stock_consolidate_df[f"{symbol}_relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df[f"{symbol}_relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df[f"{symbol}_PE_TTM_volatility_+"] * stock_consolidate_df[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df[f"{symbol}_relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df[f"{symbol}_PE_TTM_volatility_-"] * stock_consolidate_df[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df[f"{symbol}_relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df[f"{symbol}_relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df[f"{symbol}_relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{symbol}_{window_days}_price_min"] = stock_consolidate_df[symbol].min().round(2)
    stock_consolidate_df[f"{symbol}_{window_days}_price_max"] = stock_consolidate_df[symbol].max().round(2)
    stock_consolidate_df[f"{symbol}_{window_days}_price_avg"] = stock_consolidate_df[symbol].mean().round(2)
    stock_consolidate_df[f"{symbol}_{window_days}_price_std"] = np.std(stock_consolidate_df[symbol]).round(2)

    stock_consolidate_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df[f"{symbol}_relative_valuation_nextYear_projected_+"] = (stock_consolidate_df[f"{symbol}_PE_TTM_volatility_+"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df[f"{symbol}_relative_valuation_nextYear_projected_-"] = (stock_consolidate_df[f"{symbol}_PE_TTM_volatility_-"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df[f"{symbol}_relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df[f"{symbol}_relative_valuation_nextYear_projected_+"][0], stock_consolidate_df[f"{symbol}_relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{symbol}_next12months_PEG"] = PEG_12month_projected
    stock_consolidate_df[f"{symbol}_TTM_PEG"] = (stock_consolidate_df[f"{symbol}_PE_TTM"] / (((EPS_12month_projected - stock_consolidate_df[f"{symbol}_EPS_TTM"]) / stock_consolidate_df[f"{symbol}_EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df[f"{symbol}_nextYear_EPS_growthRate"] = (((EPS_12month_projected - stock_consolidate_df[f"{symbol}_EPS_TTM"]) / stock_consolidate_df[f"{symbol}_EPS_TTM"]) * 100).round(2)
    stock_consolidate_df[f"{symbol}_nextQuater_EPS_growthRate"] = (((stock_consolidate_df[f"{symbol}_EPS_latest_projected"] - stock_consolidate_df[f"{symbol}_EPS_TTM"]) / stock_consolidate_df[f"{symbol}_EPS_TTM"]) * 100).round(2)

    stock_consolidate_df[f"{symbol}_EarningYield_TTM"] = ((stock_consolidate_df[f"{symbol}_EPS_TTM"] / stock_consolidate_df[symbol]) * 100).round(2)
    stock_consolidate_df[f"{symbol}_ERP_TTM"] = stock_consolidate_df[f"{symbol}_EarningYield_TTM"] - US_T_10yrs_YTM




    # filter conditions
    conditions = [
    (stock_consolidate_df[f"{symbol}"] < stock_consolidate_df[f"{symbol}_relative_valuation_TTM_-"]),
    (stock_consolidate_df[f"{symbol}"] > stock_consolidate_df[f"{symbol}_relative_valuation_TTM_+"]),
    ((stock_consolidate_df[f"{symbol}"] >= stock_consolidate_df[f"{symbol}_relative_valuation_TTM_-"]) & (stock_consolidate_df[f"{symbol}"] <= stock_consolidate_df[f"{symbol}_relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df[f"{symbol}_curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, f"{symbol}_price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    f"{symbol}"
    ,f"{symbol}_EPS_TTM"
    ,f"{symbol}_EPS_latest_projected"
    ,f"{symbol}_PE_1yr_forward"
    ,f"{symbol}_PE_TTM"
    ,f"{symbol}_PE_TTM_avg"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_avg"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"
    ,f"{symbol}_relative_valuation_TTM_+"
    ,f"{symbol}_relative_valuation_TTM_-"
    ,f"{symbol}_relative_valuation_TTM_median"
    ,f"{symbol}_relative_valuation_nextQuater_projected_+"
    ,f"{symbol}_relative_valuation_nextQuater_projected_-"
    ,f"{symbol}_relative_valuation_nextQuater_projected_median"
    ,f"{symbol}_relative_valuation_nextYear_projected_+"
    ,f"{symbol}_relative_valuation_nextYear_projected_-"
    ,f"{symbol}_relative_valuation_nextYear_projected_median"
    ,f"{symbol}_price_valuation_assessment"
    ,f"{symbol}_nextQuater_EPS_growthRate"
    ,f"{symbol}_nextYear_EPS_growthRate"
    ,f"{symbol}_next12months_PEG"
    ,f"{symbol}_TTM_PEG"
    ,f"{symbol}_EarningYield_TTM"
    ,f"{symbol}_ERP_TTM"
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[f'{symbol}'] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[f'{symbol}'] = stock_consolidate_df[selected_cols].to_json()



# screener df will store each stock's consolidate df's first row and union them together for screening purposee
selected_cols = [col.replace(f"{symbol}_", "") for col in selected_cols]
ticker_screen_df.columns = selected_cols
# Insert rows into the DataFrame
ticker_screen_df.loc[i] = stock_consolidate_df[selected_cols].values[0]
ticker_screen_df['Ticker'] = symbol

ValueError: Length mismatch: Expected axis has 0 elements, new values have 25 elements

In [250]:
ticker_screen_df = pd.DataFrame()

In [253]:
ticker_screen_df.set_axis(
    selected_cols
    ,axis='columns'
)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 25 elements

In [252]:
ticker_screen_df

pandas.core.frame.DataFrame

In [237]:
stock_consolidate_df[selected_cols].values[0]

array([173.15, 6.97, 7.1, 19.93, 24.84, 27.01, 362.23, 605.54, 118.92,
       196.28, 180.24, 188.26, 199.94, 183.61, 191.78, 244.71, 224.72,
       234.72, 'undervalued', 1.87, 24.68, 1.1, 1.01, 4.03,
       -0.13999999999999968], dtype=object)

In [232]:
for key, value in ticker_dict_json.items():
    print(key, value)

SLB {"SLB":{"2024-07-31":48.29,"2024-07-30":47.61,"2024-07-29":47.86,"2024-07-26":48.83,"2024-07-25":48.23,"2024-07-24":48.03,"2024-07-23":49.05,"2024-07-22":49.44,"2024-07-19":49.67,"2024-07-18":48.72,"2024-07-17":48.9,"2024-07-16":48.72,"2024-07-15":47.96,"2024-07-12":46.42,"2024-07-11":46.19,"2024-07-10":45.28,"2024-07-09":45.2,"2024-07-08":46.24,"2024-07-05":46.14,"2024-07-03":47.31,"2024-07-02":46.69,"2024-07-01":46.58,"2024-06-28":47.18,"2024-06-27":47.01,"2024-06-26":46.23,"2024-06-25":47.52,"2024-06-24":47.48,"2024-06-21":45.66,"2024-06-20":45.46,"2024-06-18":44.44,"2024-06-17":43.63,"2024-06-14":43.0,"2024-06-13":43.81,"2024-06-12":44.71,"2024-06-11":44.96,"2024-06-10":44.98,"2024-06-07":44.55,"2024-06-06":43.66,"2024-06-05":43.2,"2024-06-04":43.41,"2024-06-03":44.07,"2024-05-31":45.89,"2024-05-30":45.59,"2024-05-29":45.64,"2024-05-28":46.53,"2024-05-24":46.24,"2024-05-23":46.32,"2024-05-22":46.48,"2024-05-21":48.29,"2024-05-20":48.23,"2024-05-17":48.59,"2024-05-16":48.42,"202

In [194]:
import requests

for symbol in ticker_symbols:


    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin'
        ,'operating_margin'
        ,'net_margin'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        print(key, value)

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities']) * 100
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital']) * 100
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity']) * 100
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities']) * 100

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities']) * 100
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital']) * 100
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities']) * 100


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    # calculating new ratios
    stock_ratios_annual_consolidate_df['ROE'] = annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']

    

symbol MSFT
annualReports [{'fiscalDateEnding': '2023-06-30', 'reportedCurrency': 'USD', 'totalAssets': '411976000000', 'totalCurrentAssets': '184257000000', 'cashAndCashEquivalentsAtCarryingValue': '34704000000', 'cashAndShortTermInvestments': '111262000000', 'inventory': '2500000000', 'currentNetReceivables': '48688000000', 'totalNonCurrentAssets': '232219000000', 'propertyPlantEquipment': '95641000000', 'accumulatedDepreciationAmortizationPPE': '68251000000', 'intangibleAssets': '77252000000', 'intangibleAssetsExcludingGoodwill': '9366000000', 'goodwill': '67886000000', 'investments': '95599000000', 'longTermInvestments': '9879000000', 'shortTermInvestments': '76558000000', 'otherCurrentAssets': '21807000000', 'otherNonCurrentAssets': '30601000000', 'totalLiabilities': '205753000000', 'totalCurrentLiabilities': '104149000000', 'currentAccountsPayable': '18095000000', 'deferredRevenue': '107626000000', 'currentDebt': '5247000000', 'shortTermDebt': '0', 'totalNonCurrentLiabilities': '

In [175]:
# 90% accurate
annual_income_df[income_ratio_cols + ['fiscalDateEnding']]  

,gross_margin,operating_margin,net_margin,fiscalDateEnding
13,80.162922,38.566673,30.023686,2010-06-30
12,77.729008,38.833050,33.098380,2011-06-30
11,76.221803,29.519960,23.029448,2012-06-30
10,73.814693,34.379375,28.083855,2013-06-30
9,68.816003,31.968261,25.421211,2014-06-30
8,64.695448,19.406925,13.029493,2015-06-30
7,64.038879,28.608728,22.532198,2016-06-30
6,64.522476,30.055607,26.394052,2017-06-30
5,67.156927,32.696648,15.454851,2018-06-30
4,67.154946,34.786024,31.774566,2019-06-30


In [168]:
# not accurate, cannot be used
annual_balancesheet_df[balancesheet_ratio_cols + ['fiscalDateEnding']]

,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,fiscalDateEnding
15,0.001278,-78250000000,-7.588498,333.002883,NaN,2008-12-31
14,976.481836,4584000000,134.249564,299.635236,1026.959847,2009-12-31
13,1555.080214,5442000000,134.490996,314.743867,NaN,2010-12-31
12,749.427481,3403000000,236.849838,327.237815,NaN,2011-12-31
11,318.015013,2614000000,553.634277,313.060421,NaN,2012-12-31
10,1321.518987,3860000000,162.202073,318.664301,NaN,2013-12-31
9,506.737968,3803000000,69.681830,315.034627,NaN,2014-12-31
8,616.727717,4232000000,55.529301,324.544453,NaN,2015-12-31
7,569.603524,4264000000,45.731707,331.699755,NaN,2016-12-31
6,586.725664,4400000000,32.295455,336.066748,NaN,2017-12-31


In [176]:
# # List of columns to plot
# # columns_to_plot = qtr_income_df.columns[1:]  # Exclude the 'fiscalDateEnding' column

# # Create bar charts for each column
# for column in qtr_income_ratio_cols:
#     fig = px.bar(qtr_income_df, x='fiscalDateEnding', y=column, title=column)
#     fig.show()

# Appendix

In [5]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

In [6]:
# Pivot the DataFrame
pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
pivot_sector_ticker_df = pivot_sector_ticker_df.set_index('Sector').T

pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBWI, BBY, BKNG, BWA, ...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, ACGL, AJG, AIZ,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, BIO, TECH...","[MMM, AOS, ALLE, AAL, AME, ADP, AXON, BA, BR, ...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [15]:
pivot_sector_ticker_df['Financials'].iloc[0]

['AFL',
 'ALL',
 'AXP',
 'AIG',
 'AMP',
 'AON',
 'ACGL',
 'AJG',
 'AIZ',
 'BAC',
 'BK',
 'BRK.B',
 'BLK',
 'BX',
 'BRO',
 'COF',
 'CBOE',
 'SCHW',
 'CB',
 'CINF',
 'C',
 'CFG',
 'CME',
 'CPAY',
 'DFS',
 'EG',
 'FDS',
 'FIS',
 'FITB',
 'FI',
 'BEN',
 'GPN',
 'GL',
 'GS',
 'HIG',
 'HBAN',
 'ICE',
 'IVZ',
 'JKHY',
 'JPM',
 'KEY',
 'KKR',
 'L',
 'MTB',
 'MKTX',
 'MMC',
 'MA',
 'MET',
 'MCO',
 'MS',
 'MSCI',
 'NDAQ',
 'NTRS',
 'PYPL',
 'PNC',
 'PFG',
 'PGR',
 'PRU',
 'RJF',
 'RF',
 'SPGI',
 'STT',
 'SYF',
 'TROW',
 'TRV',
 'TFC',
 'USB',
 'V',
 'WRB',
 'WFC',
 'WTW']

In [16]:
len(pivot_sector_ticker_df['Financials'].iloc[0])

71

## Appendix

In [128]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    # 'KR', # DATA ISSUE
    'LW',
    # 'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    # 'WBA', # DATA ISSUE
    'WMT'
]

In [ ]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO',
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]